In [10]:
import pandas as pd
import numpy as np

not_in_new_df = pd.read_excel("output/papers_not_in_new.xlsx")

two_banks = ("ING", "HSBC")

two_banks_df = not_in_new_df[not_in_new_df["company_name"].isin(two_banks)][["company_name", "title", "description", "paper_url"]]

In [11]:
two_banks_df

,company_name,title,description,paper_url
754,HSBC,pos (lat2005) 275 the study of su (3) super ya...,Supersymmetric Yang-Mills quantum mechanics by...,/citations?view_op=view_citation&hl=en&user=5o...
755,HSBC,time sensitive business intelligence-big data ...,Background In the competitive data driven busi...,/citations?view_op=view_citation&hl=en&user=6d...
756,HSBC,an overview of regulations and ethics of artif...,With the rapid advancement of artificial intel...,/citations?view_op=view_citation&hl=en&user=SE...
757,HSBC,towards autonomous quantum network control,Quantum networks are going to disrupt how we p...,/citations?view_op=view_citation&hl=en&user=Yh...
758,HSBC,portfolio optimization using deep learning mod...,• A robust deep learning-based stock price pre...,/citations?view_op=view_citation&hl=en&user=_U...
759,HSBC,facial features identification and object trac...,• OpenCV is a very rich and vivid library whic...,/citations?view_op=view_citation&hl=en&user=_U...
760,HSBC,handwritten digits recognition using convoluti...,"➢ In the above figure, we have a stride value ...",/citations?view_op=view_citation&hl=en&user=_U...
761,HSBC,precise stock price prediction for robust and ...,• An LSTM network-based deep learning regressi...,/citations?view_op=view_citation&hl=en&user=_U...
762,HSBC,2022 ieee international conference on artifici...,Author Index Toggle navigation IEEE Computer S...,/citations?view_op=view_citation&hl=en&user=nL...
807,ING,maintaining and monitoring aiops models agains...,AIOps solutions enable faster discovery of fai...,/citations?view_op=view_citation&hl=en&user=KE...
